In [1]:
import pandas as pd
import numpy as np
from sodapy import Socrata
import collections;
import re;
from time import time
from collections import defaultdict

In [2]:
# Import Incident Data, downloaded from https://datasf.org/
df_18_19 = pd.DataFrame.from_csv('data/sfpd_incidents_18_19.csv')
df_03_18 = pd.DataFrame.from_csv('data/sfpd_incidents_03_18.csv')

#client = Socrata("data.sfgov.org", None)
#data = client.get("cuks-n6tp", limit = 3000000)
#df_03_18 = pd.DataFrame.from_records(data)

#data = client.get("wg3w-h783", limit = 3000000)
#df_18_19 = pd.DataFrame.from_records(data)

In [3]:
print(df_03_18.shape)
print(df_18_19.shape)

print(df_18_19.columns)
print(df_03_18.columns)

(2215024, 12)
(191216, 25)
Index([u'Incident Date', u'Incident Time', u'Incident Year',
       u'Incident Day of Week', u'Report Datetime', u'Row ID', u'Incident ID',
       u'Incident Number', u'CAD Number', u'Report Type Code',
       u'Report Type Description', u'Filed Online', u'Incident Code',
       u'Incident Category', u'Incident Subcategory', u'Incident Description',
       u'Resolution', u'Intersection', u'CNN', u'Police District',
       u'Analysis Neighborhood', u'Supervisor District', u'Latitude',
       u'Longitude', u'point'],
      dtype='object')
Index([u'Category', u'Descript', u'DayOfWeek', u'Date', u'Time', u'PdDistrict',
       u'Resolution', u'Address', u'X', u'Y', u'Location', u'PdId'],
      dtype='object')


In [4]:
## Data cleaning
df_18_19 = df_18_19.drop(['Incident Year',
                          'Report Type Code',
                          'Filed Online', 
                          'Supervisor District', 
                          'incident year'])

df_18_19.columns = map(str.lower, df_18_19.columns)
df_03_18.columns = map(str.lower, df_03_18.columns)

df_18_19 = df_18_19.rename(columns={
        'incident date': 'incident_date', 
        'incident time': 'incident_time',
        'incident day of week': 'incident_dow',
        'report Datetime': 'report_datetime',
        'row id': 'row_id',
        'incident number': 'incident_number',
        'incident id': 'incident_id',
        'cad number': 'cad_number',
        'report type description': 'report_type',
        'incident code': 'incident_code',
        'incident category': 'incident_category',
        'incident subcategory': 'incident_subcategory',
        'incident description': 'incident_desc',
        'police district': 'police_district',
        'analysis neighborhood': 'neighborhood',
        'latitude': 'lat',
        'longitude': 'long'})

df_03_18 = df_03_18.rename(columns={
        'date': 'incident_date', 
        'time': 'incident_time',
        'dayofweek': 'incident_dow',
        'pdid': 'row_id',
        'incidntnum': 'incident_number',
        'incident code': 'incident_code',
        'category': 'incident_category',
        'descript': 'incident_desc',
        'pddistrict': 'police_district',
        'address': 'intersection',
        'location': 'point',
        'x': 'lat',
        'y': 'long'})


In [5]:
print(df_03_18.shape)
print(df_18_19.shape)

print(df_18_19.columns)
print(df_03_18.columns)

(2215024, 12)
(191216, 25)
Index([u'incident_date', u'incident_time', u'incident year', u'incident_dow',
       u'report datetime', u'row_id', u'incident_id', u'incident_number',
       u'cad_number', u'report type code', u'report_type', u'filed online',
       u'incident_code', u'incident_category', u'incident_subcategory',
       u'incident_desc', u'resolution', u'intersection', u'cnn',
       u'police_district', u'neighborhood', u'supervisor district', u'lat',
       u'long', u'point'],
      dtype='object')
Index([u'incident_category', u'incident_desc', u'incident_dow',
       u'incident_date', u'incident_time', u'police_district', u'resolution',
       u'intersection', u'lat', u'long', u'point', u'row_id'],
      dtype='object')


In [6]:
## Format Dates, dealing with different formats
# 1) create new column temp_date and store there datetimes recognized using first format
#df_03_18['temp_date'] = pd.to_datetime(df_03_18['incident_date'], format='%Y/%m/%d', errors='coerce')
# 2) get not recognized rows
#not_recongized = df_03_18[df_03_18.temp_date.isnull()]
# 3) inplace fill not recognized rows with values recognized using second format
#df_03_18['temp_date'].fillna(pd.to_datetime(not_recongized['incident_date'], format='%Y/%m/%d', errors='coerce'), inplace=True)
# 4) delete starttime column
#del df_03_18['incident_date']
# 5) inplace rename temp_starttime to starttime
#df_03_18.rename(columns={"temp_date": "incident_date"}, inplace=True)

In [7]:
df_03_18['incident_date'] = pd.to_datetime(df_03_18['incident_date'],format='%m/%d/%Y') 


In [12]:
## Format Dates, dealing with different formats
# 1) create new column temp_date and store there datetimes recognized using first format
df_18_19['temp_date'] = pd.to_datetime(df_18_19['incident_date'], format='%Y/%m/%d', errors='coerce')
# 2) get not recognized rows
not_recongized = df_18_19[df_18_19.temp_date.isnull()]
# 3) inplace fill not recognized rows with values recognized using second format
df_18_19['temp_date'].fillna(pd.to_datetime(not_recongized['incident_date'], format='%Y/%m/%d', errors='coerce'), inplace=True)

# 4) delete starttime column
del df_18_19['incident_date']
# 5) inplace rename temp_starttime to starttime
df_18_19.rename(columns={"temp_date": "incident_date"}, inplace=True)

df_18_19['incident_date'] = pd.to_datetime(df_18_19['incident_date'],format='%m/%d/%Y') 


In [15]:
## Merge incident data
print(len(df_03_18))
print(len(df_18_19))
df_incident = pd.concat([df_03_18, df_18_19])

print(df_incident.dtypes)
print(df_incident.head(5))


2215024
191216
cad_number                     float64
cnn                            float64
filed online                    object
incident year                  float64
incident_category               object
incident_code                  float64
incident_date           datetime64[ns]
incident_desc                   object
incident_dow                    object
incident_id                    float64
incident_number                float64
incident_subcategory            object
incident_time                   object
intersection                    object
lat                            float64
long                           float64
neighborhood                    object
point                           object
police_district                 object
report datetime                 object
report type code                object
report_type                     object
resolution                      object
row_id                           int64
supervisor district            float64
dtype: obj

In [16]:
df_incident.to_csv('data/full_incident_data.csv',index=False)